In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import cross_validate
#!pip install  imbalanced-learn
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import ColumnTransformer
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import make_pipeline
import sklearn
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv("final_office.csv")
X = df.drop(['Unnamed: 0', 'imdb_rating', 'Other_director', 'Other_writer'], axis = 1)
y = df['imdb_rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [3]:
def tidy_scores(score_dict):
    df = pd.DataFrame(score_dict)
    df.loc['mean'] = df.mean()
    df.loc['sd'] = df.std()
    df.rename({"test_score":"val_score"}, axis=1, inplace=True)
    df.index.name = "fold"
    return df.round(2)

In [4]:
def model_fit(m, X, y, plot = False):
    """Returns the mean squared error, root mean squared error and R^2 value of a fitted model based 
    on provided X and y values.
    
    Args:
        m: sklearn model object
        X: model matrix to use for prediction
        y: outcome vector to use to calculating rmse and residuals
        plot: boolean value, should fit plots be shown 
    """
    
    y_hat = m.predict(X)
    MSE = mean_squared_error(y, y_hat)
    RMSE = np.sqrt(mean_squared_error(y, y_hat))
    Rsqr = r2_score(y, y_hat)
    
    Metrics = (round(MSE, 4), round(RMSE, 4), round(Rsqr, 4))
    
    res = pd.DataFrame(
        data = {'y': y, 'y_hat': y_hat, 'resid': y - y_hat}
    )
    
    if plot:
        plt.figure(figsize=(12, 6))
        
        plt.subplot(121)
        sns.lineplot(x='y', y='y_hat', color="grey", data =  pd.DataFrame(data={'y': [min(y),max(y)], 'y_hat': [min(y),max(y)]}))
        sns.scatterplot(x='y', y='y_hat', data=res).set_title("Actual vs Fitted plot")
        
        plt.subplot(122)
        sns.scatterplot(x='y_hat', y='resid', data=res).set_title("Fitted vs Residual plot")
        plt.hlines(y=0, xmin=np.min(y), xmax=np.max(y), linestyles='dashed', alpha=0.3, colors="black")
        
        plt.subplots_adjust(left=0.0)
        
        plt.suptitle("Model (MSE, RMSE, Rsqr) = " + str(Metrics), fontsize=14)
        plt.show()
    
    return MSE, RMSE, Rsqr

In [5]:
def get_coefs(m):
    """Returns the model coefficients from a Scikit-learn model object as an array,
    includes the intercept if available.
    """
    
    # If pipeline, use the last step as the model
    if (isinstance(m, sklearn.pipeline.Pipeline)):
        m = m.steps[-1][1]
    
    
    if m.intercept_ is None:
        return m.coef_
    
    return np.concatenate([[m.intercept_], m.coef_])

# 1 Unscaled data

In [7]:
# Description of different models via pipeline
linear_pipe1 = LinearRegression()
knn_pipe1 = KNeighborsRegressor(n_neighbors=3)
lasso_pipe1 = linear_model.Lasso(alpha = 0)
ridge_pipe1 = linear_model.Ridge(alpha = 60)

# Collection of all models
model_dict = {
    'LinearRegression': linear_pipe1,
    'KNN': knn_pipe1,
    'Lasso': lasso_pipe1,
    'Ridge': ridge_pipe1}

In [8]:
kf = KFold(n_splits = 5, shuffle = True, random_state=42)

In [9]:
# Implementing the 5-fold cross validation for each model in a loop
for i, model_name in enumerate(model_dict): 
    scores = cross_validate(model_dict[model_name], X_train, y_train, cv = kf, return_train_score = True)
    scores_df = tidy_scores(scores)
    scores_df['model'] = model_name
    scores_df = scores_df.set_index("model", append=True)
    scores_df = scores_df.swaplevel()
    
    if i == 0:
        all_scores = scores_df
    else:
        all_scores = pd.concat([all_scores, scores_df], axis=0)


D:\anoconda\lib\site-packages\sklearn\model_selection\_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
D:\anoconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
D:\anoconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.723e+00, tolerance: 3.713e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
D:\ano

In [10]:
# Mean values for the train and valid scores        
display(all_scores.xs('mean', level=1, drop_level=False))
# Std Dev. values for the train and valid scores
display(all_scores.xs('sd', level=1, drop_level=False))

,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,mean,0.12,0.01,0.05,0.49
KNN,mean,0.01,0.16,0.17,0.53
Lasso,mean,0.02,0.01,0.05,0.49
Ridge,mean,0.03,0.01,0.13,0.43


,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,sd,0.22,0.01,0.22,0.02
KNN,sd,0.00,0.12,0.11,0.02
Lasso,sd,0.01,0.00,0.22,0.02
Ridge,sd,0.02,0.00,0.13,0.02


In [11]:
# find Ridge's optimal alpha
alphas = np.linspace(0, 60, num=200)
gs = GridSearchCV(
    make_pipeline(
    linear_model.Ridge()),
    param_grid = {'ridge__alpha': alphas},
    cv = KFold(5, shuffle = True, random_state = 42),
    scoring = "neg_root_mean_squared_error"
)

gs.fit(X_train, y_train)
print(gs.best_params_)

{'ridge__alpha': 9.045226130653266}


In [12]:
best_ridge1 = 9.045226130653266

In [13]:
# find lasso's optimal alpha
alphas = np.logspace(-3, 1, num=200)
gs = GridSearchCV(
    make_pipeline(
    linear_model.Lasso()),
    param_grid = {'lasso__alpha': alphas},
    cv = KFold(5, shuffle = True, random_state = 42),
    scoring = "neg_root_mean_squared_error"
)

gs.fit(X_train, y_train)
print(gs.best_params_)

{'lasso__alpha': 0.7149428986597577}


In [14]:
best_lasso1 = 0.7149428986597577

In [16]:
# model with optimal alpha
# Description of different models via pipeline

linear_pipe1o = LinearRegression()
knn_pipe1o = KNeighborsRegressor(n_neighbors=3)
lasso_pipe1o = linear_model.Lasso(alpha = best_lasso1)
ridge_pipe1o = linear_model.Ridge(alpha = best_ridge1)


# Collection of all models
model_dict = {
    'LinearRegression': linear_pipe1o,
    'KNN': knn_pipe1o,
    'Lasso': lasso_pipe1o,
    'Ridge': ridge_pipe1o}

In [17]:
# Implementing the 5-fold cross validation for each model in a loop
for i, model_name in enumerate(model_dict): 
    scores = cross_validate(model_dict[model_name], X_train, y_train, cv = kf, return_train_score = True)
    scores_df = tidy_scores(scores)
    scores_df['model'] = model_name
    scores_df = scores_df.set_index("model", append=True)
    scores_df = scores_df.swaplevel()
    
    if i == 0:
        all_scores = scores_df
    else:
        all_scores = pd.concat([all_scores, scores_df], axis=0)

In [18]:
# score of optimal model
# Mean values for the train and valid scores        
display(all_scores.xs('mean', level=1, drop_level=False))
# Std Dev. values for the train and valid scores
display(all_scores.xs('sd', level=1, drop_level=False))

,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,mean,0.01,0.01,0.05,0.49
KNN,mean,0.01,0.08,0.17,0.53
Lasso,mean,0.01,0.00,0.27,0.36
Ridge,mean,0.01,0.01,0.13,0.46


,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,sd,0.01,0.00,0.22,0.02
KNN,sd,0.01,0.01,0.11,0.02
Lasso,sd,0.01,0.00,0.08,0.01
Ridge,sd,0.00,0.00,0.14,0.02


# 2 Scaled data

##  2.1 Power_transform

In [19]:
num_index = list(range(0,X.shape[1]))
del num_index[20:30]
factor_index = list(range(20,30))

In [21]:
X.iloc[:,factor_index].head()

,Greg Daniels,Jeffrey Blitz,Ken Kwapis,Ken Whittingham,Paul Feig,B.J. Novak,Gene Stupnitsky;Lee Eisenberg,Greg Daniels.1,Michael Schur,Paul Lieberstein
0,0,0,1,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0


In [22]:
power = PowerTransformer()

trans_processing = ColumnTransformer([
    ("power_transformer", power, num_index),
    ('factor', 'passthrough', factor_index)
])

# Description of different models via pipeline
linear_pipe = Pipeline(steps = [("trans_processing", trans_processing),("model", LinearRegression())])
knn_pipe = Pipeline(steps = [("trans_processing", trans_processing), ("model", KNeighborsRegressor(n_neighbors=3))])
lasso_pipe = Pipeline(steps = [("trans_processing", trans_processing),("model", linear_model.Lasso(alpha = 0.02))])
ridge_pipe = Pipeline(steps = [("trans_processing", trans_processing), ("model", linear_model.Ridge(alpha = 60))])

# Collection of all models
model_dict = {
    'LinearRegression': linear_pipe,
    'KNN': knn_pipe,
    'Lasso': lasso_pipe,
    'Ridge': ridge_pipe}

In [23]:
# Implementing the 5-fold cross validation for each model in a loop
for i, model_name in enumerate(model_dict): 
    scores = cross_validate(model_dict[model_name], X_train, y_train, cv = kf, return_train_score = True)
    scores_df = tidy_scores(scores)
    scores_df['model'] = model_name
    scores_df = scores_df.set_index("model", append=True)
    scores_df = scores_df.swaplevel()
    
    if i == 0:
        all_scores = scores_df
    else:
        all_scores = pd.concat([all_scores, scores_df], axis=0)

In [24]:
# Mean values for the train and valid scores        
display(all_scores.xs('mean', level=1, drop_level=False))
# Std Dev. values for the train and valid scores
display(all_scores.xs('sd', level=1, drop_level=False))

,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,mean,0.12,0.01,0.01,0.51
KNN,mean,0.08,0.10,0.09,0.54
Lasso,mean,0.11,0.01,0.21,0.43
Ridge,mean,0.09,0.01,0.21,0.42


,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,sd,0.03,0.00,0.36,0.03
KNN,sd,0.01,0.03,0.16,0.02
Lasso,sd,0.04,0.01,0.10,0.02
Ridge,sd,0.01,0.00,0.11,0.02


In [25]:
X_train_pow = trans_processing.fit_transform(X_train)

In [26]:
# find Ridge's optimal alpha
alphas = np.linspace(0, 60, num=200)
gs = GridSearchCV(
    make_pipeline(
    linear_model.Ridge()),
    param_grid = {'ridge__alpha': alphas},
    cv = KFold(5, shuffle = True, random_state = 42),
    scoring = "neg_root_mean_squared_error"
)

gs.fit(X_train_pow, y_train)
print(gs.best_params_)

{'ridge__alpha': 60.0}


In [27]:
best_ridge = 60.0

In [28]:
# find lasso's optimal alpha
alphas = np.logspace(-3, 1, num=200)
gs = GridSearchCV(
    make_pipeline(
    linear_model.Lasso()),
    param_grid = {'lasso__alpha': alphas},
    cv = KFold(5, shuffle = True, random_state = 42),
    scoring = "neg_root_mean_squared_error"
)

gs.fit(X_train_pow, y_train)
print(gs.best_params_)

{'lasso__alpha': 0.04247571552536898}


In [29]:
best_lasso = 0.04247571552536898

In [30]:
# Description of different models via pipeline
linear_pipe = Pipeline(steps = [("trans_processing", trans_processing),("model", LinearRegression())])
knn_pipe = Pipeline(steps = [("trans_processing", trans_processing), ("model", KNeighborsRegressor(n_neighbors=3))])
lasso_pipe = Pipeline(steps = [("trans_processing", trans_processing),("model", linear_model.Lasso(alpha = best_lasso))])
ridge_pipe = Pipeline(steps = [("trans_processing", trans_processing), ("model", linear_model.Ridge(alpha = best_ridge))])


# Collection of all models
model_dict = {
    'LinearRegression': linear_pipe,
    'KNN': knn_pipe,
    'Lasso': lasso_pipe,
    'Ridge': ridge_pipe}

for i, model_name in enumerate(model_dict): 
    scores = cross_validate(model_dict[model_name], X_train, y_train, cv = kf, return_train_score = True)
    scores_df = tidy_scores(scores)
    scores_df['model'] = model_name
    scores_df = scores_df.set_index("model", append=True)
    scores_df = scores_df.swaplevel()
    
    if i == 0:
        all_scores = scores_df
    else:
        all_scores = pd.concat([all_scores, scores_df], axis=0)

In [31]:
# Mean values for the train and valid scores        
display(all_scores.xs('mean', level=1, drop_level=False))
# Std Dev. values for the train and valid scores
display(all_scores.xs('sd', level=1, drop_level=False))

,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,mean,0.08,0.01,0.01,0.51
KNN,mean,0.07,0.10,0.09,0.54
Lasso,mean,0.09,0.01,0.23,0.39
Ridge,mean,0.10,0.01,0.21,0.42


,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,sd,0.01,0.01,0.36,0.03
KNN,sd,0.01,0.01,0.16,0.02
Lasso,sd,0.02,0.00,0.06,0.02
Ridge,sd,0.01,0.00,0.11,0.02


## 2.3.1 Power_transform with total vote

In [40]:
dff = pd.read_csv("the_office.csv")
Xv = X.copy()
Xv['total_votes'] = dff['total_votes']
Xv_train, Xv_test, y_train, y_test = train_test_split(Xv, y, test_size = 0.2, random_state = 42)

In [43]:
num_index = list(range(0,Xv.shape[1]))
del num_index[20:30]
factor_index = list(range(20,30))
Xv.iloc[:,factor_index].head()

,Greg Daniels,Jeffrey Blitz,Ken Kwapis,Ken Whittingham,Paul Feig,B.J. Novak,Gene Stupnitsky;Lee Eisenberg,Greg Daniels.1,Michael Schur,Paul Lieberstein
0,0,0,1,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0


In [44]:
trans_processing = ColumnTransformer([
    ("power_transformer", power, num_index),
    ('factor', 'passthrough', factor_index)
])


In [45]:
Xv_train_pow = trans_processing.fit_transform(Xv_train)

In [46]:
# find Ridge's optimal alpha
alphas = np.linspace(0, 60, num=200)
gs = GridSearchCV(
    make_pipeline(
    linear_model.Ridge()),
    param_grid = {'ridge__alpha': alphas},
    cv = KFold(5, shuffle = True, random_state = 42),
    scoring = "neg_root_mean_squared_error"
)

gs.fit(Xv_train_pow, y_train)
print(gs.best_params_)

{'ridge__alpha': 11.758793969849245}


In [47]:
best_ridge = 11.758793969849245

In [48]:
# find lasso's optimal alpha
alphas = np.logspace(-3, 1, num=200)
gs = GridSearchCV(
    make_pipeline(
    linear_model.Lasso()),
    param_grid = {'lasso__alpha': alphas},
    cv = KFold(5, shuffle = True, random_state = 42),
    scoring = "neg_root_mean_squared_error"
)

gs.fit(Xv_train_pow, y_train)
print(gs.best_params_)

{'lasso__alpha': 0.04247571552536898}


In [49]:
best_lasso = 0.04247571552536898


In [50]:
trans_processing = ColumnTransformer([
    ("power_transformer", power, num_index),
    ('factor', 'passthrough', factor_index)
])


# Description of different models via pipeline
linear_pipe = Pipeline(steps = [("trans_processing", trans_processing),("model", LinearRegression())])
knn_pipe = Pipeline(steps = [("trans_processing", trans_processing), ("model", KNeighborsRegressor(n_neighbors=3))])
lasso_pipe = Pipeline(steps = [("trans_processing", trans_processing),("model", linear_model.Lasso(alpha = best_lasso))])
ridge_pipe = Pipeline(steps = [("trans_processing", trans_processing), ("model", linear_model.Ridge(alpha = best_ridge))])

# Collection of all models
model_dict = {
    'LinearRegression': linear_pipe,
    'KNN': knn_pipe,
    'Lasso': lasso_pipe,
    'Ridge': ridge_pipe}

for i, model_name in enumerate(model_dict): 
    scores = cross_validate(model_dict[model_name], Xv_train, y_train, cv = kf, return_train_score = True)
    scores_df = tidy_scores(scores)
    scores_df['model'] = model_name
    scores_df = scores_df.set_index("model", append=True)
    scores_df = scores_df.swaplevel()
    
    if i == 0:
        all_scores = scores_df
    else:
        all_scores = pd.concat([all_scores, scores_df], axis=0)

In [51]:
# Mean values for the train and valid scores        
display(all_scores.xs('mean', level=1, drop_level=False))
# Std Dev. values for the train and valid scores
display(all_scores.xs('sd', level=1, drop_level=False))

,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,mean,0.08,0.01,0.41,0.71
KNN,mean,0.07,0.09,0.29,0.62
Lasso,mean,0.07,0.01,0.55,0.62
Ridge,mean,0.09,0.01,0.50,0.68


,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,sd,0.02,0.00,0.14,0.02
KNN,sd,0.00,0.01,0.18,0.02
Lasso,sd,0.01,0.00,0.09,0.02
Ridge,sd,0.03,0.00,0.10,0.02


## 2.3.2 power trans with vote and episode

In [52]:
Xve = Xv.copy()
Xve['episode'] = dff['episode']
Xve_train, Xve_test, y_train, y_test = train_test_split(Xve, y, test_size = 0.2, random_state = 42)

In [92]:
Xve.head()

,n_lines,n_directions,n_speak_char,Angela,Dwight,Jim,Kevin,Michael,Oscar,Pam,...,Paul Feig,B.J. Novak,Gene Stupnitsky;Lee Eisenberg,Greg Daniels.1,Michael Schur,Paul Lieberstein,pos_score,neg_score,total_votes,episode
0,229,27,15,1.0,29.0,36.0,1.0,81.0,3.0,40.0,...,0,0,0,0,0,0,0.196,0.093,3706,1
1,203,20,12,4.0,17.0,24.0,8.0,75.0,13.0,12.0,...,0,1,0,0,0,0,0.240,0.076,3566,2
2,244,21,13,5.0,62.0,42.0,6.0,56.0,9.0,32.0,...,0,0,0,0,0,1,0.215,0.092,2983,3
3,243,24,14,7.0,47.0,49.0,3.0,68.0,14.0,22.0,...,0,0,0,0,1,0,0.228,0.089,2886,4
4,230,49,18,3.0,25.0,21.0,1.0,104.0,2.0,14.0,...,0,0,0,1,0,0,0.180,0.109,3179,5


In [53]:
num_index = list(range(0,Xve.shape[1]))
del num_index[20:30]
factor_index = list(range(20,30))

In [54]:
trans_processing = ColumnTransformer([
    ("power_transformer", power, num_index),
    ('factor', 'passthrough', factor_index)
])


In [55]:
Xve_train_pow = trans_processing.fit_transform(Xve_train)

In [56]:
# find Ridge's optimal alpha
alphas = np.linspace(0, 60, num=200)
gs = GridSearchCV(
    make_pipeline(
    linear_model.Ridge()),
    param_grid = {'ridge__alpha': alphas},
    cv = KFold(5, shuffle = True, random_state = 42),
    scoring = "neg_root_mean_squared_error"
)

gs.fit(Xve_train_pow, y_train)
print(gs.best_params_)

{'ridge__alpha': 11.758793969849245}


In [94]:
best_ridge = 11.758793969849245

In [57]:
# find lasso's optimal alpha
alphas = np.logspace(-3, 1, num=200)
gs = GridSearchCV(
    make_pipeline(
    linear_model.Lasso()),
    param_grid = {'lasso__alpha': alphas},
    cv = KFold(5, shuffle = True, random_state = 42),
    scoring = "neg_root_mean_squared_error"
)

gs.fit(Xve_train_pow, y_train)
print(gs.best_params_)

{'lasso__alpha': 0.036969127071950283}


In [90]:
best_lasso = 0.036969127071950283

In [59]:
trans_processing = ColumnTransformer([
    ("power_transformer", power, num_index),
    ('factor', 'passthrough', factor_index)
])


# Description of different models via pipeline
linear_pipe = Pipeline(steps = [("trans_processing", trans_processing),("model", LinearRegression())])
knn_pipe = Pipeline(steps = [("trans_processing", trans_processing), ("model", KNeighborsRegressor(n_neighbors=3))])
lasso_pipe = Pipeline(steps = [("trans_processing", trans_processing),("model", linear_model.Lasso(alpha = best_lasso))])
ridge_pipe = Pipeline(steps = [("trans_processing", trans_processing), ("model", linear_model.Ridge(alpha = best_ridge))])

# Collection of all models
model_dict = {
    'LinearRegression': linear_pipe,
    'KNN': knn_pipe,
    'Lasso': lasso_pipe,
    'Ridge': ridge_pipe}

for i, model_name in enumerate(model_dict): 
    scores = cross_validate(model_dict[model_name], Xve_train, y_train, cv = kf, return_train_score = True)
    scores_df = tidy_scores(scores)
    scores_df['model'] = model_name
    scores_df = scores_df.set_index("model", append=True)
    scores_df = scores_df.swaplevel()
    
    if i == 0:
        all_scores = scores_df
    else:
        all_scores = pd.concat([all_scores, scores_df], axis=0)

In [60]:
# Mean values for the train and valid scores        
display(all_scores.xs('mean', level=1, drop_level=False))
# Std Dev. values for the train and valid scores
display(all_scores.xs('sd', level=1, drop_level=False))

,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,mean,0.10,0.01,0.48,0.75
KNN,mean,0.08,0.09,0.32,0.63
Lasso,mean,0.09,0.01,0.59,0.66
Ridge,mean,0.13,0.01,0.57,0.72


,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,sd,0.02,0.00,0.13,0.03
KNN,sd,0.01,0.01,0.16,0.01
Lasso,sd,0.02,0.01,0.10,0.03
Ridge,sd,0.02,0.00,0.08,0.03


In [91]:
m = linear_model.Lasso(alpha = best_lasso).fit(Xve_train_pow, y_train)
get_coefs(m)

array([ 8.28648649e+00,  1.02539092e-01,  3.66078385e-02,  3.68977367e-03,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  3.42185596e-03,  0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  7.75442545e-03,  0.00000000e+00,  0.00000000e+00,
        3.54905530e-03,  0.00000000e+00,  1.70244305e-03,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  3.67092578e-01,
        7.68651860e-02,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -0.00000000e+00, -0.00000000e+00])

In [95]:
mr = linear_model.Ridge(alpha = best_ridge).fit(Xve_train_pow, y_train)
get_coefs(mr)

array([ 8.27603102e+00,  4.42719415e-02,  3.53014448e-02,  3.98719541e-02,
        6.14815548e-03,  3.72219883e-02,  2.95582096e-02, -3.01378927e-02,
        8.65040034e-02,  3.75785174e-02, -1.02699086e-02, -5.56615804e-03,
       -4.55647709e-02,  2.19227702e-02,  3.34083008e-02,  2.20810680e-02,
        2.61569012e-02,  2.80056771e-02,  8.15252966e-03,  3.83033069e-02,
       -1.38875515e-02, -3.06273947e-02, -4.91338540e-02,  3.58170264e-01,
        1.16912290e-01,  5.81056549e-02, -3.67032887e-02, -3.04422985e-02,
       -1.33370587e-02,  2.06669872e-02,  8.20747953e-02,  4.02849074e-02,
        7.29180521e-03, -2.23834027e-02,  3.40894811e-03])

In [97]:
get_coefs(m)[1:]

array([ 0.10253909,  0.03660784,  0.00368977,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.00342186,  0.        ,
        0.        , -0.        ,  0.00775443,  0.        ,  0.        ,
        0.00354906,  0.        ,  0.00170244,  0.        , -0.        ,
       -0.        , -0.        ,  0.36709258,  0.07686519,  0.        ,
       -0.        , -0.        , -0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.        , -0.        ])

In [96]:
get_coefs(mr)[1:]

array([ 0.04427194,  0.03530144,  0.03987195,  0.00614816,  0.03722199,
        0.02955821, -0.03013789,  0.086504  ,  0.03757852, -0.01026991,
       -0.00556616, -0.04556477,  0.02192277,  0.0334083 ,  0.02208107,
        0.0261569 ,  0.02800568,  0.00815253,  0.03830331, -0.01388755,
       -0.03062739, -0.04913385,  0.35817026,  0.11691229,  0.05810565,
       -0.03670329, -0.0304423 , -0.01333706,  0.02066699,  0.0820748 ,
        0.04028491,  0.00729181, -0.0223834 ,  0.00340895])

## dummy

In [67]:
Xex = df.drop(['Unnamed: 0', 'imdb_rating', 'Paul Feig','Ken Kwapis','Greg Daniels','Ken Whittingham','Jeffrey Blitz',
               'Greg Daniels.1','Gene Stupnitsky;Lee Eisenberg','Michael Schur','Paul Lieberstein','B.J. Novak'], axis = 1)
Xex_train, Xex_test, y_train, y_test = train_test_split(Xex, y, test_size = 0.2, random_state = 42)

In [68]:
Xex.head()

,n_lines,n_directions,n_speak_char,Angela,Dwight,Jim,Kevin,Michael,Oscar,Pam,...,Toby,Meredith,Darryl,Creed,Andy,Erin,Other_director,Other_writer,pos_score,neg_score
0,229,27,15,1.0,29.0,36.0,1.0,81.0,3.0,40.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.196,0.093
1,203,20,12,4.0,17.0,24.0,8.0,75.0,13.0,12.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0.240,0.076
2,244,21,13,5.0,62.0,42.0,6.0,56.0,9.0,32.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0,0,0.215,0.092
3,243,24,14,7.0,47.0,49.0,3.0,68.0,14.0,22.0,...,4.0,10.0,0.0,0.0,0.0,0.0,1,0,0.228,0.089
4,230,49,18,3.0,25.0,21.0,1.0,104.0,2.0,14.0,...,0.0,0.0,15.0,0.0,0.0,0.0,0,0,0.180,0.109


In [69]:
num_index = list(range(0,Xex.shape[1]))
del num_index[20:22]
factor_index = list(range(20,22))

In [70]:
Xex.iloc[:,factor_index].head()

,Other_director,Other_writer
0,0,1
1,0,0
2,0,0
3,1,0
4,0,0


In [71]:
trans_processing = ColumnTransformer([
    ("power_transformer", power, num_index),
    ('factor', 'passthrough', factor_index)
])

In [72]:
Xex_train_pow = trans_processing.fit_transform(Xex_train)

In [73]:
# find Ridge's optimal alpha
alphas = np.linspace(0, 60, num=200)
gs = GridSearchCV(
    make_pipeline(
    linear_model.Ridge()),
    param_grid = {'ridge__alpha': alphas},
    cv = KFold(5, shuffle = True, random_state = 42),
    scoring = "neg_root_mean_squared_error"
)

gs.fit(Xex_train_pow, y_train)
print(gs.best_params_)

{'ridge__alpha': 57.28643216080401}


In [75]:
best_ridge = 57.28643216080401

In [74]:
# find lasso's optimal alpha
alphas = np.logspace(-3, 1, num=200)
gs = GridSearchCV(
    make_pipeline(
    linear_model.Lasso()),
    param_grid = {'lasso__alpha': alphas},
    cv = KFold(5, shuffle = True, random_state = 42),
    scoring = "neg_root_mean_squared_error"
)

gs.fit(Xex_train_pow, y_train)
print(gs.best_params_)

{'lasso__alpha': 0.02221946860939524}


In [76]:
best_lasso = 0.02221946860939524

In [77]:
trans_processing = ColumnTransformer([
    ("power_transformer", power, num_index),
    ('factor', 'passthrough', factor_index)
])


# Description of different models via pipeline
linear_pipe = Pipeline(steps = [("trans_processing", trans_processing),("model", LinearRegression())])
knn_pipe = Pipeline(steps = [("trans_processing", trans_processing), ("model", KNeighborsRegressor(n_neighbors=3))])
lasso_pipe = Pipeline(steps = [("trans_processing", trans_processing),("model", linear_model.Lasso(alpha = best_lasso))])
ridge_pipe = Pipeline(steps = [("trans_processing", trans_processing), ("model", linear_model.Ridge(alpha = best_ridge))])

# Collection of all models
model_dict = {
    'LinearRegression': linear_pipe,
    'KNN': knn_pipe,
    'Lasso': lasso_pipe,
    'Ridge': ridge_pipe}

for i, model_name in enumerate(model_dict): 
    scores = cross_validate(model_dict[model_name], Xex_train, y_train, cv = kf, return_train_score = True)
    scores_df = tidy_scores(scores)
    scores_df['model'] = model_name
    scores_df = scores_df.set_index("model", append=True)
    scores_df = scores_df.swaplevel()
    
    if i == 0:
        all_scores = scores_df
    else:
        all_scores = pd.concat([all_scores, scores_df], axis=0)

In [78]:
# Mean values for the train and valid scores        
display(all_scores.xs('mean', level=1, drop_level=False))
# Std Dev. values for the train and valid scores
display(all_scores.xs('sd', level=1, drop_level=False))

,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,mean,0.09,0.01,0.14,0.48
KNN,mean,0.09,0.12,0.10,0.55
Lasso,mean,0.10,0.02,0.24,0.45
Ridge,mean,0.20,0.02,0.23,0.43


,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,sd,0.03,0.00,0.20,0.02
KNN,sd,0.03,0.04,0.17,0.03
Lasso,sd,0.05,0.02,0.09,0.02
Ridge,sd,0.05,0.01,0.10,0.02


In [80]:
Xexa = Xex.copy()
Xexa['episode'] = dff['episode']
Xexa['total_votes'] = dff['total_votes']
Xexa_train, Xexa_test, y_train, y_test = train_test_split(Xexa, y, test_size = 0.2, random_state = 42)

In [81]:
num_index = list(range(0,Xexa.shape[1]))
del num_index[20:22]
factor_index = list(range(20,22))

In [82]:
trans_processing = ColumnTransformer([
    ("power_transformer", power, num_index),
    ('factor', 'passthrough', factor_index)
])

In [83]:
Xexa_train_pow = trans_processing.fit_transform(Xexa_train)

In [84]:
# find Ridge's optimal alpha
alphas = np.linspace(0, 60, num=200)
gs = GridSearchCV(
    make_pipeline(
    linear_model.Ridge()),
    param_grid = {'ridge__alpha': alphas},
    cv = KFold(5, shuffle = True, random_state = 42),
    scoring = "neg_root_mean_squared_error"
)

gs.fit(Xexa_train_pow, y_train)
print(gs.best_params_)

{'ridge__alpha': 11.457286432160803}


In [86]:
best_ridge = 11.457286432160803

In [85]:
# find lasso's optimal alpha
alphas = np.logspace(-3, 1, num=200)
gs = GridSearchCV(
    make_pipeline(
    linear_model.Lasso()),
    param_grid = {'lasso__alpha': alphas},
    cv = KFold(5, shuffle = True, random_state = 42),
    scoring = "neg_root_mean_squared_error"
)

gs.fit(Xexa_train_pow, y_train)
print(gs.best_params_)

{'lasso__alpha': 0.036969127071950283}


In [87]:
best_lasso = 0.036969127071950283

In [88]:
trans_processing = ColumnTransformer([
    ("power_transformer", power, num_index),
    ('factor', 'passthrough', factor_index)
])


# Description of different models via pipeline
linear_pipe = Pipeline(steps = [("trans_processing", trans_processing),("model", LinearRegression())])
knn_pipe = Pipeline(steps = [("trans_processing", trans_processing), ("model", KNeighborsRegressor(n_neighbors=3))])
lasso_pipe = Pipeline(steps = [("trans_processing", trans_processing),("model", linear_model.Lasso(alpha = best_lasso))])
ridge_pipe = Pipeline(steps = [("trans_processing", trans_processing), ("model", linear_model.Ridge(alpha = best_ridge))])

# Collection of all models
model_dict = {
    'LinearRegression': linear_pipe,
    'KNN': knn_pipe,
    'Lasso': lasso_pipe,
    'Ridge': ridge_pipe}

for i, model_name in enumerate(model_dict): 
    scores = cross_validate(model_dict[model_name], Xexa_train, y_train, cv = kf, return_train_score = True)
    scores_df = tidy_scores(scores)
    scores_df['model'] = model_name
    scores_df = scores_df.set_index("model", append=True)
    scores_df = scores_df.swaplevel()
    
    if i == 0:
        all_scores = scores_df
    else:
        all_scores = pd.concat([all_scores, scores_df], axis=0)

In [89]:
# Mean values for the train and valid scores        
display(all_scores.xs('mean', level=1, drop_level=False))
# Std Dev. values for the train and valid scores
display(all_scores.xs('sd', level=1, drop_level=False))

,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,mean,0.10,0.01,0.55,0.73
KNN,mean,0.09,0.11,0.32,0.63
Lasso,mean,0.10,0.01,0.59,0.66
Ridge,mean,0.18,0.03,0.57,0.71


,,fit_time,score_time,val_score,train_score
model,fold,,,,
LinearRegression,sd,0.04,0.01,0.08,0.03
KNN,sd,0.03,0.03,0.15,0.01
Lasso,sd,0.06,0.01,0.10,0.03
Ridge,sd,0.14,0.02,0.07,0.03
